# Summary
In this notebook, we define a tensorflow model using keras.

# Modules

In [1]:
import os

import pandas as pd

import tensorflow as tf

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Working directory
We set the working directory to the project folder (currently is in the notebooks folder)

In [2]:
current_directory = os.getcwd()
if os.path.basename(current_directory) == 'notebooks':
    os.chdir('./..')

# Data import

In [3]:
all_data_ph_pangles = pd.read_parquet('./data/all_data_ph_pangles.parquet')

train_extended = all_data_ph_pangles[all_data_ph_pangles['origin'] != 'test']
test_kaggle = all_data_ph_pangles[all_data_ph_pangles['origin'] == 'test'].reset_index(drop=True)

## Data split
In this case, we only define a network training with a simple split, and in future work we will define a K-fold training.

In [4]:
target = 'price'
train_columns = ['carat', 'table', 'avg_girdle_diameter', 'new_x', 'new_y', 'new_z',
                 'new_depth', 'aspect_ratio', 'cubic_volume', 'diagonal_length',
                 'perfect_depth', 'perfect_table', 'table_x', 'table_y', 'cut_num',
                 'clarity_pos1', 'clarity_pos2', 'clarity_code_num', 'color_num', 'colorless',
                 'vt', 'crown_height_opt', 'girdle_height_opt', 'pavilion_height_opt', 'crown_angle',
                 'pavilion_angle', 'culet_angle']

x_train, x_test, y_train, y_test = train_test_split(train_extended[train_columns], train_extended[target], test_size=0.2, random_state=7)

## Data preprocessing

In [5]:
scaler_train = MinMaxScaler(clip=True)
x_train_scaled = scaler_train.fit_transform(x_train)
x_test_scaled = scaler_train.transform(x_test)
test_kaggle_scaled = scaler_train.transform(test_kaggle[train_columns])

# Model definition

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024, activation='relu', input_shape=(x_train_scaled.shape[1],)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='mse',
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              28672     
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 717,825
Trainable params: 717,825
Non-trainable params: 0
_________________________________________________________________


# Model training

In [7]:
callback = tf.keras.callbacks.EarlyStopping(patience=10)
model.fit(
    x_train_scaled,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=callback,
    verbose=1
)

Epoch 1/100
4411/4411 [==============================] - 21s 5ms/step - loss: 2858011.2500 - root_mean_squared_error: 1690.5653 - val_loss: 622015.6875 - val_root_mean_squared_error: 788.6797
Epoch 2/100
4411/4411 [==============================] - 21s 5ms/step - loss: 569279.5625 - root_mean_squared_error: 754.5062 - val_loss: 547589.5625 - val_root_mean_squared_error: 739.9929
Epoch 3/100
4411/4411 [==============================] - 21s 5ms/step - loss: 519873.2812 - root_mean_squared_error: 721.0224 - val_loss: 504948.0000 - val_root_mean_squared_error: 710.5969
Epoch 4/100
4411/4411 [==============================] - 24s 5ms/step - loss: 484089.4062 - root_mean_squared_error: 695.7653 - val_loss: 482988.2500 - val_root_mean_squared_error: 694.9736
Epoch 5/100
4411/4411 [==============================] - 23s 5ms/step - loss: 456274.5312 - root_mean_squared_error: 675.4810 - val_loss: 475949.9375 - val_root_mean_squared_error: 689.8912
Epoch 6/100
4411/4411 [=========================

# Model evaluation

In [8]:
test_predictions = model.predict(x_test_scaled)
model_loss = mean_squared_error(y_test, test_predictions, squared=False)
print(f'Model NN: Loss {model_loss:.4f}')

1379/1379 [==============================] - 3s 2ms/step
Model NN: Loss 600.9551


# Kaggle predictions

In [9]:
test_kaggle['price'] = model.predict(test_kaggle_scaled)
test_kaggle['price'].to_csv(f'./outputs/base_keras_nn.csv', index_label='id')

4033/4033 [==============================] - 8s 2ms/step
